##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 분산 입력

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/input"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/input.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

The [tf.distribute](https://www.tensorflow.org/guide/distributed_training) APIs provide an easy way for users to scale their training from a single machine to multiple machines. When scaling their model, users also have to distribute their input across multiple devices. `tf.distribute` provides APIs using which you can automatically distribute your input across devices.

이 가이드는 `tf.distribute` API를 사용하여 분산 데이터세트 및 반복기를 생성할 수 있는 다양한 방법을 보여줍니다. 또한 다음 주제들을 다룹니다.

- `tf.distribute.Strategy.experimental_distribute_dataset` 및 `tf.distribute.Strategy.experimental_distribute_datasets_from_function`의 사용법, 샤딩 및 배치 처리 옵션
- 분산 데이터세트를 반복할 수 있는 다양한 방법
- `tf.distribute.Strategy.experimental_distribute_dataset` / `tf.distribute.Strategy.experimental_distribute_datasets_from_function` API와 `tf.data` API의 차이점과 사용자가 사용하는 데 있어 발생할 수 있는 제한 사항

This guide does not cover usage of distributed input with Keras APIs.

## 분산 데이터세트

`tf.distribute` API를 사용하여 조정하려면 사용자가 `tf.data.Dataset`을 사용하여 입력을 나타내는 것이 좋습니다. `tf.distribute`는 `tf.data.Dataset`(예: 각 가속기 기기에 데이터 자동 프리페치)에서 효율적으로 작동하도록 만들어졌으며 성능 최적화가 구현에 정기적으로 통합되었습니다. `tf.data.Dataset` 외에 다른 사용에 대한 사용 사례가 있는 경우 나중에 이 가이드의 [섹션](%22tensorinputs%22)을 참고하세요. 비분산 훈련 루프에서 사용자는 먼저 `tf.data.Dataset` 인스턴스를 만든 다음 요소를 반복합니다. 예를 들면 다음과 같습니다.


In [ ]:
# Import TensorFlow
!pip install tf-nightly
import tensorflow as tf

# Helper libraries
import numpy as np
import os

print(tf.__version__)

In [ ]:
global_batch_size = 16
# Create a tf.data.Dataset object.
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

# Iterate over the dataset using the for..in construct.
for inputs in dataset:
  print(train_step(inputs))


사용자가 존재하는 코드를 최소한으로 변경하면서 `tf.distribute` 전략을 사용할 수 있도록 `tf.data.Dataset` 인스턴스를 배포하고 분산 데이터세트 객체를 반환하는 두 개의 API가 도입되었습니다. 그런 다음 사용자는 이 분산 데이터세트 인스턴스를 반복하고 이전과 같이 모델을 훈련할 수 있습니다. 이제 두 가지 API `tf.distribute.Strategy.experimental_distribute_dataset` 및 `tf.distribute.Strategy.experimental_distribute_datasets_from_function`를 자세히 살펴보겠습니다.

### `tf.distribute.Strategy.experimental_distribute_dataset`

#### 사용법

This API takes a `tf.data.Dataset` instance as input and returns a `tf.distribute.DistributedDataset` instance. You should batch the input dataset with a value that is equal to the global batch size. This global batch size is the number of samples that you want to process across all devices in 1 step. You can iterate over this distributed dataset in a Pythonic fashion or create an iterator using `iter`. The returned object is not a `tf.data.Dataset` instance and does not support any other APIs that transform or inspect the dataset in any way. This is the recommended API if you don’t have specific ways in which you want to shard your input over different replicas.


In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(global_batch_size)
# Distribute input using the `experimental_distribute_dataset`.
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
# 1 global batch of data fed to the model in 1 step.
print(next(iter(dist_dataset)))

#### Properties

##### 배치 처리

`tf.distribute`는 입력된 `tf.data.Dataset` 인스턴스를 전역 배치 크기와 동기화된 복제본 수로 나눈 새 배치 크기로 다시 배치합니다. 동기화 중인 복제본의 수는 훈련 중에 그래디언트 올리듀스(allreduce)에 참여하는 기기의 수와 같습니다. 사용자가 분산 반복기에서 `next`를 호출하면 복제본마다 배치 크기의 데이터가 각 복제본에 반환됩니다. 다시 배치된 데이터세트 카디널리티는 항상 여러 복제본의 배수입니다. 다음은 몇 가지 예입니다.

- `tf.data.Dataset.range(6).batch(4, drop_remainder=False)`

    - Without distribution:

        - Batch 1: [0, 1, 2, 3]
        - Batch 2: [4, 5]

    - With distribution over 2 replicas. The last batch ([4, 5]) is split between 2 replicas.

        - Batch 1:
            - Replica 1:[0, 1]
            - Replica 2:[2, 3]
        - Batch 2:
            - Replica 2: [4]
            - Replica 2: [5]

- `tf.data.Dataset.range(4).batch(4)`

    - Without distribution:
        - Batch 1: [[0], [1], [2], [3]]
    - With distribution over 5 replicas:
        - Batch 1:
            - Replica 1: [0]
            - Replica 2: [1]
            - Replica 3: [2]
            - Replica 4: [3]
            - Replica 5: []

- `tf.data.Dataset.range(8).batch(4)`

    - Without distribution:
        - Batch 1: [0, 1, 2, 3]
        - Batch 2: [4, 5, 6, 7]
    - With distribution over 3 replicas:
        - Batch 1:
            - Replica 1: [0, 1]
            - Replica 2: [2, 3]
            - Replica 3: []
        - Batch 2:
            - Replica 1: [4, 5]
            - Replica 2: [6, 7]
            - Replica 3: []

Note: The above examples only illustrate how a global batch is split on different replicas. It is not advisable to depend on the actual values that might end up on each replica as it can change depending on the implementation.

Rebatching the dataset has a space complexity that increases linearly with the number of replicas. This means that for the multi worker training use case the input pipeline can run into OOM errors. 

##### Sharding

`tf.distribute`는 또한 다중 작업자 훈련에서 입력 데이터세트를 자동 샤딩합니다. 작업자의 CPU 기기에 각 데이터세트가 생성됩니다. 작업자 집합에 대한 데이터세트를 자동 샤딩하면 각 작업자에게 전체 데이터세트의 하위 세트가 할당됩니다(올바른 `tf.data.experimental.AutoShardPolicy`가 설정된 경우). 이는 각 스텝에서 겹치지 않는 데이터세트 요소의 전역 배치 크기가 각 작업자에 의해 처리되도록 합니다. 자동 샤딩에는 `tf.data.experimental.DistributeOptions`을 사용해 지정할 수 있는 몇 개의 다양한 옵션이 있습니다.

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
dataset = dataset.with_options(options)

`tf.data.experimental.AutoShardPolicy`에 대해서 세 가지 다른 옵션을 설정할 수 있습니다.

- AUTO: This is the default option which means an attempt will be made to shard by FILE. The attempt to shard by FILE fails if a file-based dataset is not detected. `tf.distribute` will then fall back to sharding by DATA. Note that if the input dataset is file-based but the number of files is less than the number of workers, an `InvalidArgumentError` will be raised. If this happens, explicitly set the policy to `AutoShardPolicy.DATA`, or split your input source into smaller files such that number of files is greater than number of workers.

- FILE: This is the option if you want to shard the input files over all the workers. You should use this option if the number of input files is much larger than the number of workers and the data in the files is evenly distributed. The downside of this option is having idle workers if the data in the files is not evenly distributed. If the number of files is less than the number of workers, an `InvalidArgumentError` will be raised. If this happens, explicitly set the policy to `AutoShardPolicy.DATA`. For example, let us distribute 2 files over 2 workers with 1 replica each. File 1 contains [0, 1, 2, 3, 4, 5] and File 2 contains [6, 7, 8, 9, 10, 11]. Let the total number of replicas in sync be 2 and global batch size be 4.

    - Worker 0:
        - Batch 1 =  Replica 1: [0, 1]
        - Batch 2 =  Replica 1: [2, 3]
        - Batch 3 =  Replica 1: [4]
        - Batch 4 =  Replica 1: [5]
    - Worker 1:
        - Batch 1 =  Replica 2: [6, 7]
        - Batch 2 =  Replica 2: [8, 9]
        - Batch 3 =  Replica 2: [10]
        - Batch 4 =  Replica 2: [11]

- DATA: This will autoshard the elements across all the workers. Each of the workers will read the entire dataset and only process the shard assigned to it. All other shards will be discarded. This is generally used if the number of input files is less than the number of workers and you want better sharding of data across all workers. The downside is that the entire dataset will be read on each worker. For example, let us distribute 1 files over 2 workers. File 1 contains [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]. Let the total number of replicas in sync be 2.

    - Worker 0:
        - Batch 1 =  Replica 1: [0, 1]
        - Batch 2 =  Replica 1: [4, 5]
        - Batch 3 =  Replica 1: [8, 9]
    - Worker 1:
        - Batch 1 =  Replica 2: [2, 3]
        - Batch 2 =  Replica 2: [6, 7]
        - Batch 3 =  Replica 2: [10, 11]

- OFF: If you turn off autosharding, each worker will process all the data. For example, let us distribute 1 files over 2 workers. File 1 contains [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]. Let the total number of replicas in sync be 2. Then each worker will see the following distribution:

    - Worker 0:

        - Batch 1 =  Replica 1: [0, 1]
        - Batch 2 =  Replica 1: [2, 3]
        - Batch 3 =  Replica 1: [4, 5]
        - Batch 4 =  Replica 1: [6, 7]
        - Batch 5 =  Replica 1: [8, 9]
        - Batch 6 =  Replica 1: [10, 11]

    - Worker 1:

        - Batch 1 =  Replica 2: [0, 1]
        - Batch 2 =  Replica 2: [2, 3]
        - Batch 3 =  Replica 2: [4, 5]
        - Batch 4 =  Replica 2: [6, 7]
        - Batch 5 =  Replica 2: [8, 9]
        - Batch 6 =  Replica 2: [10, 11] 

##### 프리페치

기본적으로 `tf.distribute`는 사용자 제공 `tf.data.Dataset` 인스턴스의 끝에 프리페치 변환을 추가합니다. 프리페치 변환에 대한 인수 `buffer_size`는 동기화 중인 복제본의 수와 같습니다.

### `tf.distribute.Strategy.experimental_distribute_datasets_from_function`

#### 사용법

이 API는 입력 함수를 받고 `tf.distribute.DistributedDataset` 인스턴스를 반환합니다. 사용자가 전달하는 입력 함수는`tf.distribute.InputContext` 인수를 갖고 `tf.data.Dataset` 인스턴스를 반환해야 합니다. 이 API를 사용하면 `tf.distribute`는 더 이상 입력 함수로부터 반환된 사용자의 `tf.data.Dataset` 인스턴스를 변경하지 않습니다. 데이터세트를 배치하고 샤딩하는 것은 사용자가 해야 합니다. `tf.distribute`는 작업자 각각의 CPU 기기에서 입력 함수를 호출합니다. 사용자가 자체 배치 처리 및 샤딩 로직을 지정할 수 있게 해주는 것 이외에도, 다중 작업자 훈련을 사용할 때 `tf.distribute.Strategy.experimental_distribute_dataset`과 비교하여 확장성과 성능이 더 우수합니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def dataset_fn(input_context):
  batch_size = input_context.get_per_replica_batch_size(global_batch_size)
  dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(64).batch(16)
  dataset = dataset.shard(
    input_context.num_input_pipelines, input_context.input_pipeline_id)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(2) # This prefetches 2 batches per device.
  return dataset

dist_dataset = mirrored_strategy.experimental_distribute_datasets_from_function(dataset_fn)

#### Properties

##### 배치 처리

입력 함수의 반환 값인 `tf.data.Dataset` 인스턴스는 복제본별 배치 크기를 사용하여 배치해야 합니다. 복제본별 배치 크기는 전역 배치 크기를 동기화 훈련에 참여하는 복제본의 수로 나눈 값입니다. `tf.distribute`가 각 작업자의 CPU 기기에서 입력 함수를 호출하기 때문입니다. 지정된 작업자에서 생성된 데이터세트는 해당 작업자의 모든 복제본에서 사용할 수 있어야 합니다. 

##### Sharding

사용자의 입력 함수에 대한 인수로 암시적으로 전달되는 `tf.distribute.InputContext` 객체는 내부에서 `tf.distribute` 에 의해 생성됩니다. 작업자 수, 현재 작업자 ID 등에 대한 정보가 있습니다.이 입력 함수는 `tf.distribute.InputContext` 오브젝트의 일부인 이러한 특성을 사용하여 사용자가 설정 한 정책에 따라 샤딩을 처리 할 수 있습니다.


##### 프리페치

`tf.distribute` 는 사용자가 제공한 입력 함수가 반환한 `tf.data.Dataset` 의 끝에 프리페치 변환을 추가하지 않습니다.

참고 : `tf.distribute.Strategy.experimental_distribute_dataset` 및 `tf.distribute.Strategy.experimental_distribute_datasets_from_function` **은 `tf.data.Dataset` 유형이 아닌 `tf.distribute.DistributedDataset` 인스턴스를** 반환합니다. Distributed Iterators 섹션에 표시된 대로 이러한 인스턴스를 반복하고 `element_spec` 속성을 사용할 수 있습니다. 

## 분산 반복기

비분산 `tf.data.Dataset` 인스턴스와 유사하게, `tf.distribute.DistributedDataset`에서 요소에 접근하여 반복하려면 `tf.distribute.DistributedDataset` 인스턴스에 반복기를 생성해야 합니다. 다음은 `tf.distribute.DistributedIterator`를 생성하고 모델을 훈련할 때 사용하는 방법입니다.


### Usages

#### Use a Pythonic for loop construct

You can use a user friendly Pythonic loop to iterate over the `tf.distribute.DistributedDataset`. The elements returned from the `tf.distribute.DistributedIterator` can be a single `tf.Tensor` or a `tf.distribute.DistributedValues` which contains a value per replica. Placing the loop inside a `tf.function` will give a performance boost. However, `break` and `return` are currently not supported for a loop over a `tf.distribute.DistributedDataset` that is placed inside of a `tf.function`. We also don't support placing the loop inside a `tf.function` when using multi-worker strategies such as `tf.distribute.experimental.MultiWorkerMirroredStrategy` and `tf.distribute.TPUStrategy`. Placing the loop inside `tf.function` works for single worker `tf.distribute.TPUStrategy` but not when using TPU pods.

In [ ]:
global_batch_size = 16
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function
def train_step(inputs):
  features, labels = inputs
  return labels - 0.3 * features

for x in dist_dataset:
  # train_step trains the model using the dataset elements
  loss = mirrored_strategy.run(train_step, args=(x,))
  print("Loss is ", loss)

#### `iter`를 사용하여 명시적인 반복기 만들기

`tf.distribute.DistributedDataset` 인스턴스의 요소를 반복하기 위해 `iter` API를 사용하여 `tf.distribute.DistributedIterator`를 생성할 수 있습니다. 명시적인 반복기를 사용하면 고정된 수의 스텝을 반복할 수 있습니다. `tf.distribute.DistributedIterator` 인스턴스 `dist_iterator`에서 다음 요소를 가져오려면 `next(dist_iterator)`, `dist_iterator.get_next()` 또는 `dist_iterator.get_next_as_optional()`을 호출할 수 있습니다. 앞의 두 개는 본질적으로 동일합니다.

In [ ]:
num_epochs = 10
steps_per_epoch = 5
for epoch in range(num_epochs):
  dist_iterator = iter(dist_dataset)
  for step in range(steps_per_epoch):
    # train_step trains the model using the dataset elements
    loss = mirrored_strategy.run(train_step, args=(next(dist_iterator),))
    # which is the same as
    # loss = mirrored_strategy.run(train_step, args=(dist_iterator.get_next(),))
    print("Loss is ", loss)

`next()` 또는 `tf.distribute.DistributedIterator.get_next()`를 사용하여 `tf.distribute.DistributedIterator`의 끝에 도달하면 OutOfRange 오류가 발생합니다. 클라이언트는 Python 측에서 오류를 포착하고 체크포인트 및 평가와 같은 다른 작업을 계속할 수 있습니다. 그러나 호스트 훈련 루프를 사용하는 경우(예: `tf.function`당 여러 스텝 실행) 다음과 같이 작동하지 않습니다.

```
@tf.function
def train_fn(iterator):
  for _ in tf.range(steps_per_loop):
    strategy.run(step_fn, args=(next(iterator),))
```

`train_fn`은 스텝 본문을 `tf.range`안에 배치하여 여러 스텝을 래핑합니다. 이 경우 종속성이 없는 루프에서 다른 반복이 병렬로 시작될 수 있으므로 이전 반복 계산이 완료되기 전에 이후 반복에서 OutOfRange 오류가 트리거될 수 있습니다. OutOfRange 오류가 발생하면 함수의 모든 op가 즉시 종료됩니다. 이런 경우를 피하려면 OutOfRange 오류를 발생시키지 않는 대안은 `tf.distribute.DistributedIterator.get_next_as_optional()`입니다. `get_next_as_optional`은 다음 요소를 포함하거나 `tf.distribute.DistributedIterator`가 끝에 도달한 경우 값이 없는 `tf.experimental.Optional`을 반환합니다.

In [ ]:
# You can break the loop with get_next_as_optional by checking if the Optional contains value
global_batch_size = 4
steps_per_loop = 5
strategy = tf.distribute.MirroredStrategy(devices=["GPU:0", "CPU:0"])

dataset = tf.data.Dataset.range(9).batch(global_batch_size)
distributed_iterator = iter(strategy.experimental_distribute_dataset(dataset))

@tf.function
def train_fn(distributed_iterator):
  for _ in tf.range(steps_per_loop):
    optional_data = distributed_iterator.get_next_as_optional()
    if not optional_data.has_value():
      break
    per_replica_results = strategy.run(lambda x:x, args=(optional_data.get_value(),))
    tf.print(strategy.experimental_local_results(per_replica_results))
train_fn(distributed_iterator)

## Using `element_spec` property

분산된 데이터세트의 요소를 `tf.function`으로 전달하여 `tf.TypeSpec` 보장을 원할 경우, `tf.function`의 `input_signature` 인수를 지정합니다. 분산 데이터세트의 출력은 `tf.distribute.DistributedValues`이며 단일 기기 또는 여러 기기에 대한 입력을 나타낼 수 있습니다. 이 분산 값에 해당하는 `tf.TypeSpec`을 가져오려면 분산 데이터세트 또는 분산 반복기 객체의 `element_spec` 속성을 사용할 수 있습니다.

In [ ]:
global_batch_size = 16
epochs = 5
steps_per_epoch = 5
mirrored_strategy = tf.distribute.MirroredStrategy()

dataset = tf.data.Dataset.from_tensors(([1.],[1.])).repeat(100).batch(global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

@tf.function(input_signature=[dist_dataset.element_spec])
def train_step(per_replica_inputs):
  def step_fn(inputs):
    return 2 * inputs
  
  return mirrored_strategy.run(step_fn, args=(per_replica_inputs,))

for _ in range(epochs):
  iterator = iter(dist_dataset)
  for _ in range(steps_per_epoch):
    output = train_step(next(iterator))
    tf.print(output)

## Partial Batches

사용자가 생성한 `tf.data.Dataset` 인스턴스에 복제본의 수로 균등하게 나눌 수 없는 배치 크기가 포함되어 있거나 데이터세트 인스턴스의 카디널리티가 배치 크기로 나눌 수 없는 경우 부분 배치가 발생합니다. 이는 데이터세트가 여러 복제본에 분산될 때 일부 반복기에 대한 `next` 호출로 OutOfRangeError가 발생함을 의미합니다. 이 사용 사례를 처리하기 위해 `tf.distribute`는 처리할 데이터가 더 이상 없는 복제본에서 배치 크기가 0인 더미 배치를 반환합니다.


단일 작업자 사례의 경우 반복기에서 `next` 호출로 데이터가 반환되지 않으면 배치 크기가 0인 더미 배치가 작성되어 데이터세트의 실제 데이터와 함께 사용됩니다. 부분 배치의 경우 마지막 전역 배치 데이터에는 더미 배치 데이터와 함께 실제 데이터가 포함됩니다. 데이터 처리를 위한 중지 조건이 이제 복제본에 데이터가 있는지 확인합니다. 복제본에 데이터가 없으면 OutOfRange 오류가 발생합니다.

For the multi worker case, the boolean value representing presence of data on each of the workers is aggregated using cross replica communication and this is used to identify if all the workers have finished processing the distributed dataset. Since this involves cross worker communication there is some performance penalty involved.


## 경고 사항

* When using `tf.distribute.Strategy.experimental_distribute_dataset` APIs with a multiple worker setup, users pass a `tf.data.Dataset` that reads from files. If the `tf.data.experimental.AutoShardPolicy` is set to `AUTO` or `FILE`, the actual per step batch size may be smaller than the user defined global batch size. This can happen when the remaining elements in the file are less than the global batch size. Users can either exhaust the dataset without depending on the number of steps to run or set  `tf.data.experimental.AutoShardPolicy` to `DATA` to work around it.

* Stateful dataset transformations are currently not supported with `tf.distribute` and any stateful ops that the dataset may have are currently ignored. For example, if your dataset has a `map_fn` that uses `tf.random.uniform` to rotate an image, then you have a dataset graph that depends on state (i.e the random seed) on the local machine where the python process is being executed.

* Experimental `tf.data.experimental.OptimizationOptions` that are disabled by default can in certain contexts -- such as when used together with `tf.distribute` -- cause a performance degradation. You should only enable them after you validate that they benefit the performance of your workload in a distribute setting.


- `tf.distribute.experimental_distribute_dataset` 또는 `tf.distribute.experimental_distribute_datasets_from_function`을 사용할 때 작업자에 의해 처리되는 데이터의 순서는 보장되지 않습니다. 일반적으로 `tf.distribute`을 사용하여 예측을 조정하는 경우 요구됩니다. 하지만 배치의 각 요소에 인덱스를 삽입하고 그에 맞게 출력을 정렬하면 됩니다. 다음은 출력을 정렬하는 방법에 대한 예제 코드 조각입니다.

Note: `tf.distribute.MirroredStrategy()` is used here for the sake of convenience. We only need to reorder inputs when we are using multiple workers and `tf.distribute.MirroredStrategy` is used to distribute training on a single worker.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
dataset_size = 24
batch_size = 6
dataset = tf.data.Dataset.range(dataset_size).enumerate().batch(batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

def predict(index, inputs):
  outputs = 2 * inputs
  return index, outputs

result = {}
for index, inputs in dist_dataset:
  output_index, outputs = mirrored_strategy.run(predict, args=(index, inputs))
  indices = list(mirrored_strategy.experimental_local_results(output_index))
  rindices = []
  for a in indices:
    rindices.extend(a.numpy())
  outputs = list(mirrored_strategy.experimental_local_results(outputs))
  routputs = []
  for a in outputs:
    routputs.extend(a.numpy())
  for i, value in zip(rindices, routputs):
    result[i] = value

print(result)

<a name="tensorinputs"> # 표준 tf.data.Dataset 인스턴스를 사용하지 않는 경우 데이터를 어떻게 배포하나요? </a>

때때로 사용자가 `tf.data.Dataset`을 사용하여 입력을 나타내고 이후에 언급한 API를 사용하여 데이터 세트를 여러 기기에 분배할 수 없습니다. 이런 경우 생성기의 원시 텐서 또는 입력을 사용할 수 있습니다.

### Use experimental_distribute_values_from_function for arbitrary tensor inputs

`strategy.run`은 `next(iterator)`의 출력인 `tf.distribute.DistributedValues`를 허용합니다. 텐서 값을 전달하려면 `experimental_distribute_values_from_function`을 사용하여 원시 텐서에서 `tf.distribute.DistributedValues`를 구성합니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
worker_devices = mirrored_strategy.extended.worker_devices

def value_fn(ctx):
  return tf.constant(1.0)

distributed_values = mirrored_strategy.experimental_distribute_values_from_function(value_fn)
for _ in range(4):
  result = mirrored_strategy.run(lambda x:x, args=(distributed_values,))
  print(result)

### Use tf.data.Dataset.from_generator if your input is from a generator

If you have a generator function that you want to use, you can create a `tf.data.Dataset` instance using the `from_generator` API.

참고: 현재 `tf.distribute.TPUStrategy`에서는 지원하지 않습니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
def input_gen():
  while True:
    yield np.random.rand(4)

# use Dataset.from_generator
dataset = tf.data.Dataset.from_generator(
    input_gen, output_types=(tf.float32), output_shapes=tf.TensorShape([4]))
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)
iterator = iter(dist_dataset)
for _ in range(4):
  mirrored_strategy.run(lambda x:x, args=(next(iterator),))